In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [2]:
!pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from dotenv import load_dotenv

load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

In [6]:
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

In [7]:
df = pd.read_sql('select * from trending_videos', engine)

In [8]:
df.head()

,video_id,title,channel,published_at,category_id,view_count,like_count,comment_count
0,-4GmbBoYQjE,I Survived 100 Hours In An Ancient Temple,MrBeast,2025-05-10 16:00:01+00:00,24,29883658,1320076,40438
1,OjTNQNr7LA8,Trump Mother's Day Cold Open - SNL,Saturday Night Live,2025-05-11 04:18:21+00:00,23,942175,35368,2474
2,InxXnmZhjKQ,Outer Banks - First Impressions 🇺🇸,Peter Santenello,2025-05-10 15:00:24+00:00,19,570382,19566,2162
3,__Yrwoxz-4Y,I Faced My Biggest Fear! (Plastic Surgery),The Anazala Family,2025-05-10 15:04:40+00:00,24,1910494,28508,5178
4,MUc5VUtpGl8,AMP MARIO KART IRL,AMP,2025-05-10 19:00:48+00:00,24,1017702,42383,2151


In [9]:
df.drop_duplicates(subset=['video_id'], inplace=True)

In [10]:
df['published_at'] = pd.to_datetime(df['published_at'])

In [11]:
df['published_at']

0    2025-05-10 16:00:01+00:00
1    2025-05-11 04:18:21+00:00
2    2025-05-10 15:00:24+00:00
3    2025-05-10 15:04:40+00:00
4    2025-05-10 19:00:48+00:00
5    2025-05-10 14:30:33+00:00
6    2025-05-09 01:00:07+00:00
7    2025-05-09 16:05:08+00:00
8    2025-05-09 12:01:39+00:00
9    2025-05-09 14:37:29+00:00
10   2025-05-10 14:01:27+00:00
11   2025-05-10 10:45:07+00:00
12   2025-05-10 05:23:02+00:00
13   2025-05-10 02:00:40+00:00
14   2025-05-09 01:00:33+00:00
15   2025-05-09 20:00:06+00:00
16   2025-05-09 16:00:06+00:00
17   2025-05-10 02:13:22+00:00
18   2025-05-09 21:00:19+00:00
19   2025-05-10 03:08:55+00:00
20   2025-05-08 16:00:19+00:00
21   2025-05-10 14:00:07+00:00
22   2025-05-09 20:28:02+00:00
23   2025-04-29 14:51:48+00:00
24   2025-05-09 21:17:14+00:00
25   2025-05-08 19:42:59+00:00
26   2025-05-09 13:01:27+00:00
27   2025-05-09 15:01:01+00:00
28   2025-05-09 20:00:01+00:00
29   2025-05-09 18:28:50+00:00
30   2025-05-09 15:52:24+00:00
31   2025-05-09 14:00:06+00:00
32   202

In [12]:
df['video_age_days'] = (pd.Timestamp.now(tz='UTC') - df['published_at']).dt.days

In [13]:
df['engagement_percent'] = ((df['comment_count'] + df['like_count']) / df['view_count']) * 100

In [14]:
df['engagement_percent'] = df['engagement_percent'].round(2)

In [15]:
df.nlargest(10, 'view_count')[['title', 'view_count']]

,title,view_count
43,Grand Theft Auto VI Trailer 2,100750053
0,I Survived 100 Hours In An Ancient Temple,29883658
20,The Conjuring: Last Rites | Official Teaser,16254006
47,I Built Minecraft In Real Life!,8762691
48,MY 2ND DEEPEST FEAR.. There's a GIANT Spider [...,4280750
25,LIVE: Robert Prevost named Pope Leo XIV,2864920
21,TRAPPED in ABANDONED SCHOOL *Security Cameras*,2823978
6,Coulda Been Records BALTIMORE Auditions hosted...,2726907
32,First American pope elected,2509936
7,Peacemaker Season 2 | Official Teaser | Max,2248546


In [16]:
df.nlargest(10, 'engagement_percent')[['title', 'engagement_percent']]

,title,engagement_percent
26,The all-knowing lie detector | Chris & Jack,13.04
17,How GTA 6 REALLY got their graphics that good,12.78
41,$UICIDEBOY$ - Self-Inflicted (Lyric Video),12.70
48,MY 2ND DEEPEST FEAR.. There's a GIANT Spider [...,10.29
30,Avril Lavigne - Young & Dumb (feat. Simple Pla...,9.43
9,How I Became A Pro Bodybuilder,8.93
16,Kocky Ka - Dark Nights (Remix) Feat Meek Mill ...,7.60
27,Fixing Ludwig’s Stolen Kei Truck,6.19
43,Grand Theft Auto VI Trailer 2,6.14
39,Overwatch 2 x Street Fighter 6 | Collaboration...,5.83


In [17]:
df.to_sql('trending_videos_cleaned', con=engine, if_exists='replace', index=False)

50